In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Pull data from csv file- traffic camera locations
csv_file = "BPD_Part_1_Victim_Based_Crime_Data.csv"
baltimore_crime_df = pd.read_csv(csv_file)
baltimore_crime_df.head()

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,Total Incidents
0,09/02/2017,23:30:00,3JK,4200 AUDREY AVE,ROBBERY - RESIDENCE,I,KNIFE,913.0,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",ROW/TOWNHO,1
1,09/02/2017,23:00:00,7A,800 NEWINGTON AVE,AUTO THEFT,O,NaN,133.0,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",STREET,1
2,09/02/2017,22:53:00,9S,600 RADNOR AV,SHOOTING,Outside,FIREARM,524.0,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",Street,1
3,09/02/2017,22:50:00,4C,1800 RAMSAY ST,AGG. ASSAULT,I,OTHER,934.0,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",ROW/TOWNHO,1
4,09/02/2017,22:31:00,4E,100 LIGHT ST,COMMON ASSAULT,O,HANDS,113.0,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",STREET,1


In [4]:
crime_columns = ["CrimeDate", "CrimeTime", "Location", "Description", "Weapon", "District", "Neighborhood", "Longitude", "Latitude", "Location 1", "Total Incidents"]
baltimore_crime_df = baltimore_crime_df[crime_columns].copy()
baltimore_crime_df = baltimore_crime_df.rename(columns={"Location": "Location_Name", "Location 1":"Exact_Location", "Total Incidents": "Total_Incidents"})
baltimore_crime_df.index.names = ['ID']
baltimore_crime_df.head()

,CrimeDate,CrimeTime,Location_Name,Description,Weapon,District,Neighborhood,Longitude,Latitude,Exact_Location,Total_Incidents
ID,,,,,,,,,,,
0,09/02/2017,23:30:00,4200 AUDREY AVE,ROBBERY - RESIDENCE,KNIFE,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",1
1,09/02/2017,23:00:00,800 NEWINGTON AVE,AUTO THEFT,NaN,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",1
2,09/02/2017,22:53:00,600 RADNOR AV,SHOOTING,FIREARM,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",1
3,09/02/2017,22:50:00,1800 RAMSAY ST,AGG. ASSAULT,OTHER,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",1
4,09/02/2017,22:31:00,100 LIGHT ST,COMMON ASSAULT,HANDS,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",1


In [5]:
baltimore_crime_df.dtypes

CrimeDate           object
CrimeTime           object
Location_Name       object
Description         object
Weapon              object
District            object
Neighborhood        object
Longitude          float64
Latitude           float64
Exact_Location      object
Total_Incidents      int64
dtype: object

In [6]:
baltimore_crime_df.dropna()
baltimore_crime_df = baltimore_crime_df[pd.notnull(baltimore_crime_df['Weapon'])]
baltimore_crime_df.head()

,CrimeDate,CrimeTime,Location_Name,Description,Weapon,District,Neighborhood,Longitude,Latitude,Exact_Location,Total_Incidents
ID,,,,,,,,,,,
0,09/02/2017,23:30:00,4200 AUDREY AVE,ROBBERY - RESIDENCE,KNIFE,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",1
2,09/02/2017,22:53:00,600 RADNOR AV,SHOOTING,FIREARM,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",1
3,09/02/2017,22:50:00,1800 RAMSAY ST,AGG. ASSAULT,OTHER,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",1
4,09/02/2017,22:31:00,100 LIGHT ST,COMMON ASSAULT,HANDS,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",1
6,09/02/2017,21:15:00,3400 HARMONY CT,HOMICIDE,FIREARM,SOUTHEASTERN,Canton,-76.56827,39.28202,"(39.2820200000, -76.5682700000)",1


In [7]:
baltimore_crime_df = baltimore_crime_df.reset_index()
baltimore_crime_df.index.names = ['Index_Number']
baltimore_crime_df = baltimore_crime_df.drop('ID', 1)
baltimore_crime_df.head()

,CrimeDate,CrimeTime,Location_Name,Description,Weapon,District,Neighborhood,Longitude,Latitude,Exact_Location,Total_Incidents
Index_Number,,,,,,,,,,,
0,09/02/2017,23:30:00,4200 AUDREY AVE,ROBBERY - RESIDENCE,KNIFE,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",1
1,09/02/2017,22:53:00,600 RADNOR AV,SHOOTING,FIREARM,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",1
2,09/02/2017,22:50:00,1800 RAMSAY ST,AGG. ASSAULT,OTHER,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",1
3,09/02/2017,22:31:00,100 LIGHT ST,COMMON ASSAULT,HANDS,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",1
4,09/02/2017,21:15:00,3400 HARMONY CT,HOMICIDE,FIREARM,SOUTHEASTERN,Canton,-76.56827,39.28202,"(39.2820200000, -76.5682700000)",1


In [22]:
baltimore_crime_df.groupby(["Neighborhood","Description"]).count()


CrimeDate  CrimeTime  Location_Name  \
Neighborhood  Description                                                 
Abell         AGG. ASSAULT                 51         51             51   
              COMMON ASSAULT               87         87             87   
              HOMICIDE                      3          3              3   
              ROBBERY - CARJACKING          2          2              2   
              ROBBERY - COMMERCIAL         26         26             26   
              ROBBERY - RESIDENCE           2          2              2   
              ROBBERY - STREET             41         41             41   
Allendale     AGG. ASSAULT                132        132            132   
              COMMON ASSAULT              222        222            222   
              HOMICIDE                     15         15             15   
              RAPE                          6          6              6   
              ROBBERY - CARJACKING          3          3              3   
              ROBBERY - COMMERCIAL          5          5              5   
              ROBBERY - RESIDENCE           5          5              5   
              ROBBERY - STREET             36         36             36   
              SHOOTING                     24         24             24   
Arcadia       AGG. ASSAULT                 32         32             32   
              COMMON ASSAULT               56         56             56   
              RAPE                          1          1              1   
              ROBBERY - CARJACKING          1          1              1   
              ROBBERY - COMMERCIAL          1          1              1   
              ROBBERY - STREET             10         10             10   
              SHOOTING                      6          6              6   
Arlington     AGG. ASSAULT                171        171            171   
              COMMON ASSAULT              228        228            228   
              HOMICIDE                     12         12             12   
              RAPE                          6          6              6   
              ROBBERY - CARJACKING          2          2              2   
              ROBBERY - COMMERCIAL         18         18             18   
              ROBBERY - RESIDENCE           3          3              3   
...                                       ...        ...            ...   
Wrenlane      HOMICIDE                      5          5              5   
              ROBBERY - CARJACKING          2          2              2   
              ROBBERY - RESIDENCE           1          1              1   
              ROBBERY - STREET              6          6              6   
              SHOOTING                      3          3              3   
Wyman Park    AGG. ASSAULT                  3          3              3   
              COMMON ASSAULT               11         11             11   
              RAPE                          1          1              1   
              ROBBERY - COMMERCIAL         32         32             32   
              ROBBERY - RESIDENCE           1          1              1   
              ROBBERY - STREET              8          8              8   
Wyndhurst     AGG. ASSAULT                  3          3              3   
              COMMON ASSAULT               21         21             21   
              ROBBERY - COMMERCIAL          1          1              1   
              ROBBERY - STREET              2          2              2   
Yale Heights  AGG. ASSAULT                 68         68             68   
              COMMON ASSAULT               97         97             97   
              HOMICIDE                      7          7              7   
              RAPE                          3          3              3   
              ROBBERY - CARJACKING          3          3              3   
              ROBBERY - RESIDENCE           4          4     

In [1]:
connection_string = "root:password@localhost/ProjectTwo_db"
engine = create_engine(f'mysql://{connection_string}')

NameError: name 'create_engine' is not defined

In [36]:
engine.table_names()

['crime_data']

In [37]:
baltimore_crime_df.to_sql(name='crime_data', con=engine, if_exists='append', index=True)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '0' for key 'PRIMARY'")
[SQL: INSERT INTO crime_data (`Index_Number`, `CrimeDate`, `CrimeTime`, `Location_Name`, `Description`, `Weapon`, `District`, `Neighborhood`, `Longitude`, `Latitude`, `Exact_Location`, `Total_Incidents`) VALUES (%(Index_Number)s, %(CrimeDate)s, %(CrimeTime)s, %(Location_Name)s, %(Description)s, %(Weapon)s, %(District)s, %(Neighborhood)s, %(Longitude)s, %(Latitude)s, %(Exact_Location)s, %(Total_Incidents)s)]
[parameters: ({'Index_Number': 0, 'CrimeDate': '09/02/2017', 'CrimeTime': '23:30:00', 'Location_Name': '4200 AUDREY AVE', 'Description': 'ROBBERY - RESIDENCE', 'Weapon': 'KNIFE', 'District': 'SOUTHERN', 'Neighborhood': 'Brooklyn', 'Longitude': -76.60540999999999, 'Latitude': 39.22951, 'Exact_Location': '(39.2295100000, -76.6054100000)', 'Total_Incidents': 1}, {'Index_Number': 1, 'CrimeDate': '09/02/2017', 'CrimeTime': '22:53:00', 'Location_Name': '600 RADNOR AV', 'Description': 'SHOOTING', '